In [ ]:
import numpy as np
from scipy.optimize import minimize

# Define the neural network architecture
class Net:
    def __init__(self):
        self.w1 = np.random.randn(2, 4)
        self.w2 = np.random.randn(4, 1)

    def forward(self, x):
        h = 1 / (1 + np.exp(-np.dot(x, self.w1)))
        y_pred = 1 / (1 + np.exp(-np.dot(h, self.w2)))
        return y_pred

    def loss(self, x, y):
        y_pred = self.forward(x)
        loss = np.mean((y - y_pred)**2)
        return loss

    def parameters(self):
        return np.concatenate([self.w1.ravel(), self.w2.ravel()])

    def set_parameters(self, params):
        self.w1 = np.reshape(params[:8], (2, 4))
        self.w2 = np.reshape(params[8:], (4, 1))

# Define the training data
x_train = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y_train = np.array([0, 1, 1, 0])

# Instantiate the neural network
net = Net()

# Define the optimization function
def optimize(params):
    net.set_parameters(params)
    return net.loss(x_train, y_train)

# Train the neural network using scipy.optimize.minimize
res = minimize(optimize, net.parameters(), method='BFGS', options={'maxiter': 500, 'disp': True})

# Print the results
print(f"Optimization status: {res.message}")
print(f"Final loss: {res.fun}")
print(f"Optimized parameters: {res.x}")

# Evaluate the neural network on the test data
x_test = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y_test = np.array([0, 1, 1, 0])
y_pred = np.round(net.forward(x_test)).astype(int)
print(f"Test accuracy: {(y_pred == y_test).sum() / len(y_test)}")

In [ ]:
import torch
from torch.autograd import Variable
from scipy.optimize import fmin_l_bfgs_b

# Define the neural network model
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(1, 10)
        self.fc2 = torch.nn.Linear(10, 1)

    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        x = self.fc2(x)
        return x

# Define the loss function
criterion = torch.nn.MSELoss()

# Define the dataset
x_data = torch.Tensor([[1.0], [2.0], [3.0], [4.0]])
y_data = torch.Tensor([[2.0], [4.0], [6.0], [8.0]])

# Initialize the neural network model
net = Net()

# Define the optimizer
def closure():
    optimizer.zero_grad()
    y_pred = net(x_data)
    loss = criterion(y_pred, y_data)
    loss.backward()
    return loss.item()

optimizer = torch.optim.LBFGS(net.parameters(), lr=0.1)

# Train the model
max_iter = 100
for i in range(max_iter):
    loss = optimizer.step(closure)
    print(f"iteration {i+1}: loss = {loss:.4f}")

# Print the learned parameters
for name, param in net.named_parameters():
    if param.requires_grad:
        print(name, param.data)

In [ ]:
import torch
from scipy.optimize import minimize
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler

# Generate synthetic regression dataset
X, y = make_regression(n_samples=100, n_features=5, noise=0.5, random_state=1)

# Scale the input data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Convert the numpy arrays to PyTorch tensors
x_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y.reshape(-1, 1), dtype=torch.float32)

# Define the neural network
net = torch.nn.Sequential(torch.nn.Linear(5, 10),
                           torch.nn.ReLU(),
                           torch.nn.Linear(10, 1))

# Define the loss function
mse_loss = torch.nn.MSELoss()

# Flatten the neural network parameters
n_params = sum(p.numel() for p in net.parameters())
params_init = torch.randn(n_params)

# Define the loss function
def loss(params):
    # Load the current set of parameters into the neural network
    with torch.no_grad():
        net_params = torch.tensor(params, dtype=torch.float32).reshape(-1, 1)
        current_param_idx = 0
        for p in net.parameters():
            p.data = net_params[current_param_idx:current_param_idx+p.numel()].reshape(p.shape)
            current_param_idx += p.numel()

    # Compute the output of the neural network for the given input
    y_pred = net(x_tensor)

    # Compute the mean squared error loss between the predicted and actual outputs
    loss = mse_loss(y_tensor, y_pred)
    return loss.item()

# Use L-BFGS-B method to optimize the parameters
options = {"maxiter": 100, "disp": True}
res = minimize(loss, params_init, method="BFGS", options=options)

# Load the optimized parameters back to the neural network
with torch.no_grad():
    net_params = torch.tensor(res.x, dtype=torch.float32).reshape(-1, 1)
    current_param_idx = 0
    for p in net.parameters():
        p.data = net_params[current_param_idx:current_param_idx+p.numel()].reshape(p.shape)
        current_param_idx += p.numel()

# Compute the final loss
final_loss = loss(res.x)
print(f"Final loss: {final_loss:.4f}")


In [ ]:
import torch
import numpy as np
from torch import nn, optim
from scipy.optimize import minimize


# Define the neural network
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(1, 5)
        self.fc2 = nn.Linear(5, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Create an instance of the neural network
net = Net()

# Create some input data
x = torch.randn(10, 1)

# Define the loss function
loss_fn = nn.MSELoss()

# Define a function to compute the loss and gradients
def fun(params):
    # Copy the parameters to the neural network
    with torch.no_grad():
        for p, param in zip(net.parameters(), params):
            p.copy_(torch.tensor(param))

    # Compute the output of the neural network
    output = net(x)

    # Compute the loss
    loss = loss_fn(output, torch.ones_like(output))

    # Compute the gradients
    net.zero_grad()
    loss.backward()
    gradients = []
    for p in net.parameters():
        gradients.append(p.grad.view(-1).numpy())

    # Return the loss and gradients
    return loss.item(), np.concatenate(gradients)

# Initialize the parameters
params0 = np.concatenate([p.view(-1).detach().numpy() for p in net.parameters()])


# Use Scipy to optimize the parameters
res = minimize(fun, params0, jac=True, method='BFGS')

# Copy the optimized parameters to the neural network
with torch.no_grad():
    for p, param in zip(net.parameters(), res.x):
        p.copy_(torch.tensor(param))

# Compute the output of the neural network
output = net(x)

# Backpropagate the gradients
output.backward(torch.ones_like(output))


In [ ]:
import torch
from scipy.optimize import minimize

# Define the neural network architecture
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc = torch.nn.Linear(1, 1)

    def forward(self, x):
#         print(x.dtype)
        x = F.relu(self.fc(x))
        return x

# Define the loss function
def loss_fn(y_pred, y_true):
    return torch.mean((y_pred - y_true) ** 2)

# Define the training data
x_train = torch.linspace(-5, 5, 100).reshape(-1, 1)
y_train = x_train ** 2

# Initialize the neural network
net = Net()

# Define the training loop
def train(params):
    # Update the neural network weights
    with torch.no_grad():
#         print(net.fc.weight.dtype)
        net.fc.weight = torch.nn.Parameter(torch.tensor(params).reshape(1, 1), requires_grad=True)
        net.double()
#         print(net.fc.weight.dtype)
        
    # Compute the predictions and loss
    y_pred = net(x_train.double())
    loss = loss_fn(y_pred, y_train)
    # Compute the gradients and update the weights
    net.zero_grad()
    loss.backward()
    grads = net.fc.weight.grad.detach().numpy().reshape(1)
    grads = grads.astype('float64') # convert to the correct data type
    return loss.item(), grads

# Define the optimizer
def optimize(params):
    loss, grads = train(params)
    return loss, grads

# Initialize the weight parameter and optimize it with scipy
params0 = net.fc.weight.detach().numpy().reshape(1)
bounds = [(None, None)]
for epoch in range(100):
    result = minimize(optimize, params0, jac=True, method='L-BFGS-B', bounds=bounds)
    params0 = result.x

# print(result)

# Update the neural network weights with the optimized value
net.fc.weight = torch.nn.Parameter(torch.tensor(result.x).reshape(1, 1))

# Test the neural network on some new data
x_test = torch.tensor([[2.0], [3.0], [4.0]])
y_pred = net(x_test.double())
print(x_test, y_pred, params0)


In [ ]:
import torch
from torch import nn
from scipy.optimize import minimize_scalar

# Define the neural network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.linear = nn.Linear(1, 1)
        
    def forward(self, x):
        return self.linear(x)

# Define the loss function
def loss_fn(output, target):
    return torch.mean((output - target) ** 2)

# Create an instance of the neural network
net = Net()

# Define the input tensor
x = torch.tensor([[1.0]])

# Define the target tensor
target = x ** 2

# Define the optimization function
def optimize_fn(x):
    net.linear.weight.data = torch.tensor([[x]])
    input_tensor = torch.tensor([[x]])
    output = net(input_tensor)
    loss = loss_fn(output, target)
    loss.backward(torch.ones_like(output))
    return loss.item()

# Train the neural network using Scipy's minimize_scalar function
res = minimize_scalar(optimize_fn)
print("Optimal weight:", res.x)
